In [3]:
# To Do
# Remove any columns and records that will not suit the investigation
# Prepare a class for the cleaned data table
# Upload the data table to a SQL database instance

In [4]:
# Importing Dependencies
import pandas as pd
import os

### Cleaning the WHO Mortality Dataset
* Contains records from 1988 to 2020 on causes of death
* Some records in the "Sex" column are unspecified, coded as a 9. For our purposes we will remove these records
* 

In [5]:

# Cycle through the different parts of the Morticd and combine them into a single dataframe
dirLoc = 'Resources' #filepath of the directory
df = pd.DataFrame()

for file in os.scandir(dirLoc):
    if "Morti" in file.path: # Only append files from the Mortality Datasets
        df = df.append(pd.read_csv(file.path,))

        
        

c:\Users\daypa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\Users\daypa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
# Removing records with value 9
df["Sex"].value_counts() # There are 20,701 records with the 9 value

# Creating new dataframe wihtout 9 values in the Sex field
df_Removed9 = df[df["Sex"] != 9]

df_Removed9["Sex"].value_counts() # 9 values removed from the original dataset

1    2358126
2    2127394
Name: Sex, dtype: int64

In [7]:
df_Removed9.head(10)

,Country,Admin1,SubDiv,Year,List,Cause,Sex,Frmat,IM_Frmat,Deaths1,...,Deaths21,Deaths22,Deaths23,Deaths24,Deaths25,Deaths26,IM_Deaths1,IM_Deaths2,IM_Deaths3,IM_Deaths4
0,1400,NaN,NaN,2001,101,1000,1,7,8,332,...,95.0,NaN,NaN,NaN,NaN,0.0,8.0,NaN,NaN,NaN
1,1400,NaN,NaN,2001,101,1000,2,7,8,222,...,112.0,NaN,NaN,NaN,NaN,0.0,11.0,NaN,NaN,NaN
2,1400,NaN,NaN,2001,101,1001,1,7,8,24,...,5.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
3,1400,NaN,NaN,2001,101,1001,2,7,8,14,...,6.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
4,1400,NaN,NaN,2001,101,1002,1,7,8,0,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
5,1400,NaN,NaN,2001,101,1002,2,7,8,0,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
6,1400,NaN,NaN,2001,101,1003,1,7,8,0,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
7,1400,NaN,NaN,2001,101,1003,2,7,8,0,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
8,1400,NaN,NaN,2001,101,1004,1,7,8,0,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
9,1400,NaN,NaN,2001,101,1004,2,7,8,0,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN


In [8]:
#Removing uneccesary columns
df_excised_cols = df_Removed9[["Country","Year","Cause","Sex","Deaths1"]]# Takes too long to process consider a big data solution?

In [9]:
df_excised_cols #Looks like the process took long due to some syntax error.

,Country,Year,Cause,Sex,Deaths1
0,1400,2001,1000,1,332
1,1400,2001,1000,2,222
2,1400,2001,1001,1,24
3,1400,2001,1001,2,14
4,1400,2001,1002,1,0
...,...,...,...,...,...
610179,5020,2020,Y870,2,1
610180,5020,2020,Y871,1,3
610181,5020,2020,Y883,2,1
610182,5020,2020,Y899,1,1


In [10]:
minYear = min(df_excised_cols['Year'])
maxYear = max(df_excised_cols['Year'])
print(minYear , maxYear)
# Observing the Scope of the Mortality Dataset  (1988 - 2020)
# Scope of the Meat Consumption ddDataset is 1990 - 2029
# We can limit the size of the datsets within the years 2000 and 2020
# Need to consider ways of improving write speed of Dataframe to PSQL database
# Try batch uploading to the SQL database

1988 2020


In [11]:
# removing all records that preced 2000
df_2000 = df_excised_cols[df_excised_cols["Year"] >= 2000]
df_2000

,Country,Year,Cause,Sex,Deaths1
0,1400,2001,1000,1,332
1,1400,2001,1000,2,222
2,1400,2001,1001,1,24
3,1400,2001,1001,2,14
4,1400,2001,1002,1,0
...,...,...,...,...,...
610179,5020,2020,Y870,2,1
610180,5020,2020,Y871,1,3
610181,5020,2020,Y883,2,1
610182,5020,2020,Y899,1,1


In [12]:
df_2000.count() # There are no null values
df_2000.dtypes # Columns Cause, Sex, and Country dont seem to be the datatype we need them to be

# We can explicitly assign the string datatype to the Cause column
df_2000['Cause'] = df_2000['Cause'].astype('string')

df_2000.dtypes # The Cause Column is now string

<ipython-input-12-a6f3d49267f9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2000['Cause'] = df_2000['Cause'].astype('string')


Country     int64
Year        int64
Cause      string
Sex         int64
Deaths1     int64
dtype: object

## WHO Country and Cause of Death Coding
The world health organization codes the causes of death and the country of origin using their mapping which is specified in tables in the Mortality dataset documentation.
I need to map the value of the codes to our clean dataset

In [17]:
# mapping the country name to the dataset using the WHO_Country_Map
# Read up on the .map() documentation in pandas

#Read in the WHO_Coutry_Map as a dataframe
country_Map_df = pd.read_csv('Resources/WHO_Country_Map.csv')

# Convert dataframe to dictionary
country_Map = country_Map_df.to_dict()

{'country': {0: 1010,
  1: 1020,
  2: 1025,
  3: 1030,
  4: 1035,
  5: 1040,
  6: 1045,
  7: 1060,
  8: 1070,
  9: 1080,
  10: 1090,
  11: 1100,
  12: 1115,
  13: 1120,
  14: 1125,
  15: 1130,
  16: 1135,
  17: 1140,
  18: 1160,
  19: 1170,
  20: 1180,
  21: 1190,
  22: 1192,
  23: 1220,
  24: 1230,
  25: 1240,
  26: 1250,
  27: 1260,
  28: 1270,
  29: 1280,
  30: 1290,
  31: 1300,
  32: 1303,
  33: 1310,
  34: 1320,
  35: 1325,
  36: 1330,
  37: 1340,
  38: 1360,
  39: 1365,
  40: 1370,
  41: 1385,
  42: 1390,
  43: 1400,
  44: 1410,
  45: 1420,
  46: 1430,
  47: 1460,
  48: 1465,
  49: 1470,
  50: 1480,
  51: 1510,
  52: 1520,
  53: 1530,
  54: 1546,
  55: 1555,
  56: 1560,
  57: 1570,
  58: 2005,
  59: 2010,
  60: 2020,
  61: 2025,
  62: 2030,
  63: 2040,
  64: 2045,
  65: 2050,
  66: 2060,
  67: 2070,
  68: 2085,
  69: 2090,
  70: 2110,
  71: 2120,
  72: 2130,
  73: 2140,
  74: 2150,
  75: 2160,
  76: 2170,
  77: 2180,
  78: 2190,
  79: 2200,
  80: 2210,
  81: 2230,
  82: 2240,
  8